In [1]:
import numpy as np 
import pandas as pd
from sklearn.svm import SVR
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import RandomizedSearchCV
%matplotlib inline

In [2]:
train = pd.read_csv("mertrain.csv")
test = pd.read_csv("mertest.csv")

# process columns, apply LabelEncoder to categorical features
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(train[c].values) + list(test[c].values)) 
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

# shape        
print('Shape train: {}\nShape test: {}'.format(train.shape, test.shape))

Shape train: (4209, 378)
Shape test: (4209, 377)


In [3]:
y = train['y'].values
y_mean = np.mean(y)
id_test = test['ID'].values
X = train.drop(['y'], axis=1).values
X_test= test.values

In [4]:
kf=KFold(n_splits=5, shuffle=True, random_state=0)
kf.get_n_splits(X)

5

In [5]:
for train_index, test_index in kf.split(X):
    X_train, y_train=X[train_index], y[train_index]
    X_valid, y_valid=X[test_index], y[test_index]

In [6]:
pipe=Pipeline([("scaler", MaxAbsScaler()), ("svm", SVR())])

In [12]:
param_distributions = {'svm__kernel': ['linear','poly','rbf','sigmoid'],
                       'svm__C': [0.001,0.01,0.1,1,10,100,1000]
                         }

In [25]:
#scaler=MinMaxScaler()
#scaler=StandardScaler()
#scaler=MaxAbsScaler()
#scalet=RobustScaler()

In [26]:
#X_train_scaled=scaler.transform(X_train)
#X_valid_scaled=scaler.transform(X_valid)

In [27]:
#param_grid={
#    'C': [0.001,0.01,0.1,1,10,100,1000]
#           }

In [13]:
rcv=RandomizedSearchCV(pipe,param_distributions, n_iter=5, scoring='r2', cv=kf, n_jobs=2, random_state=0)

In [14]:
rcv.fit(X_train, y_train)

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
          error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('scaler', MaxAbsScaler(copy=True)), ('svm', SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))]),
          fit_params=None, iid=True, n_iter=5, n_jobs=2,
          param_distributions={'svm__kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'svm__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
          pre_dispatch='2*n_jobs', random_state=0, refit=True,
          return_train_score='warn', scoring='r2', verbose=0)

In [16]:
print("R^2 valid: {: }".format(r2_score(y_valid, rcv.predict(X_valid))))
print("R^2 train: {: }".format(r2_score(y_train, rcv.predict(X_train))))

R^2 valid:  0.5620406614601142
R^2 train:  0.5519800692943185


In [42]:
y_pred=grid2.predict(X_test)

In [43]:
sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = y_pred
sub.to_csv('svr-minmaxscaler-bestparam2.csv', index=False)
